# Question Answering Portion

##### Importing libraries

In [1]:
!py -m pip install beautifulsoup4
!py -m pip install pandas
!py -m pip install matplotlib
!py -m pip install nltk
!py -m pip install numpy
!py -m pip install scikit-learn

You should consider upgrading via the 'C:\Users\Dr. Sunil Tripathi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Dr. Sunil Tripathi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Dr. Sunil Tripathi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Dr. Sunil Tripathi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Dr. Sunil Tripathi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Dr. Sunil Tripathi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


In [1]:
#for webpage and snippet extraction
import urllib
import requests
from bs4 import BeautifulSoup
from requests import get
import re
import collections
import pandas as pd
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

#for answer filtering
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True)
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Dr. Sunil
[nltk_data]     Tripathi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Dr. Sunil
[nltk_data]     Tripathi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Dr. Sunil
[nltk_data]     Tripathi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
!py -m pip install fake-useragent

You should consider upgrading via the 'C:\Users\Dr. Sunil Tripathi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


In [2]:
from fake_useragent import UserAgent

In [3]:
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

In [4]:
# For Keyword Matching
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]

### Web Page Extraction

In [5]:
def webpageextraction(query):
    ua = UserAgent()

    google_url = "https://www.google.com/search?q=" + query #+ "&num=" + str(number_result)
    response = requests.get(google_url, {"User-Agent": ua.random})
    soup = BeautifulSoup(response.text, "html.parser")

    result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})

    links = []
    titles = []
    descriptions = []
    for r in result_div:
        # Checks if each element is present, else, raise exception
        try:
            link = r.find('a', href = True)
            title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
            description = r.find('div', attrs={'class':'s3v9rd'}).get_text()

            # Check to make sure everything is present before appending
            if link != '' and title != '' and description != '': 
                links.append(link['href'])
                titles.append(title)
                descriptions.append(description)
        # Next loop if one element is not present
        except:
            continue
    to_remove = []
    clean_links = []
    for i, l in enumerate(links):
        clean = re.search('\/url\?q\=(.*)\&sa',l)

        # Anything that doesn't fit the above pattern will be removed
        if clean is None:
            to_remove.append(i)
            continue
        if "youtube.com" not in clean.group(1):
            clean_links.append(clean.group(1))

    # Remove the corresponding titles & descriptions
    for x in to_remove:
        del titles[x]
        del descriptions[x]
    return clean_links

    '''  query=query.replace(' ','+')
    URL = f"https://google.com/search?q={query}"
    ## user agent specification, as google returns different results for mobile and desktop
    ### desktop user-agent
    USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:65.0) Gecko/20100101 Firefox/65.0"
    ### mobile user-agent
    MOBILE_USER_AGENT = "Mozilla/5.0 (Linux; Android 7.0; SM-G930V Build/NRD90M) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.125 Mobile Safari/537.36"

    ## user-agent must be in the headers. We pass in a dictionary for the headers in requests.
    headers = {"user-agent" : USER_AGENT}
    response = requests.get(URL, headers=headers)
    print(response.content)
    ## checking if the request was successful (if we obtain 200 in return, it is successful), then we put it into BeautifulSoup for content parsing
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")

    ## parsing the data and extracting all anchor links from the page. storing the results in the results list.
    results = []
    for g in soup.find_all('div', class_='r'):
        anchors = g.find_all('a')
        print(anchors)
        if anchors:
            link = anchors[0]['href']
            title = g.find('h3').text
            item = {
                "title": title,
                "link": link
            }
            results.append(item['link'])
    print(results)
    return results'''

### Snippet Extraction

In [6]:
def snippetextraction(results,i):
    # Snippet Extraction from the obtained web page urls
    try:
        raw = get(results[i]).text ###this is how we can extract raw html from web pages
    except:
        return 'error604'
    html = requests.get(results[i]).content
    #1 Recoding
    unicode_str = html.decode("utf8")
    encoded_str = unicode_str.encode("ascii",'ignore')
    news_soup = BeautifulSoup(encoded_str, "html.parser")
    a_text = news_soup.find_all('p')
    #2 Removing
    y=[re.sub(r'<.+?>',r'',str(a)) for a in a_text]
    y=[x.replace('\n','') for x in y]
    y=[x.strip() for x in y]
    y=' '.join(y)
    return y

### Tokenisation

In [7]:
def tokenise(raw):
    #Tokenisation
    sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
    #print(sent_tokens)
    word_tokens = nltk.word_tokenize(raw)# converts to list of words
    return sent_tokens

### Preprocessing

In [8]:
def LemTokens(tokens):
    lemmer = WordNetLemmatizer()
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

### Generating Response

In [9]:
def response(query,sent_tokens):
    robo_response=''
    sent_tokens.append(query)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        print(robo_response)
        return 'error502'
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

### Jarvis Greeting & Bye

In [34]:
# for greeting the user
def greeting(query):
    """If user's input is a greeting, return a greeting response"""
    for word in query.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)
        
# for saying bye and ending the program
def byee():
    print("Jarvis: Bye! take care..")
    return False

#for setting flag to true and introduction
def introduce():
    #as we are taking a variable flag (true until user keeps asking questions)
    print("Jarvis: Hi! My name is Jarvis. I will answer all your queries. If you want to exit, type something like 'Bye' !")
    return True

## Final Operation

In [35]:
#Jarvis introduces itself
flag=introduce()
#site index
print('May I know your name?')
#User introduces
user_name=input()
print('Jarvis: Hello ',user_name,", How can I help you today?")

while(flag==True):
    i=0
    #taking user question
    query = input(user_name+': ')
    query=query.lower()
    
    if('bye' not in query.split(' ')):
        if(query=='thanks' or query=='thank you' ):
            flag=False
            print("Jarvis: You are welcome..")
        else:
            if(greeting(query)!=None):
                print("Jarvis: "+greeting(query))
            else:
                result=webpageextraction(query)
                raw=''
                while((not raw.strip()) or (('access' in raw.lower()) and ('denied' in raw.lower()))):
                    raw=snippetextraction(result,i)
                    if raw=='error604':
                        break
                    i+=1
                if raw=='error604':
                    print("Sorry I couldn't find an answer.")
                    continue
                if raw:
                    sent_tokens=tokenise(raw)
                else:
                    print("Sorry I couldn't find an answer.")
                print("Jarvis: ",end='')
                answer=response(query,sent_tokens)
                if answer=='error502':
                    continue
                print(answer+'\n'+result[i])
                
                
                #now checking satisfaction
                satisfied=False
                while(not satisfied):
                    print('Are you satisfied by my answer? Enter "yes" or "no".')
                    if (input().lower()=='yes'):
                        break
                    else:
                        i+=1
                        raw=snippetextraction(result,i)
                        while((not raw.strip()) or (('access' in raw.lower()) and ('denied' in raw.lower()))):
                            raw=snippetextraction(result,i)
                            i+=1
                        if raw=='error604':
                            print('Sorry, I am out of answers now.')
                            print("Anything else you would like to ask?")
                            break
                        if raw:
                            sent_tokens=tokenise(raw)
                        else:
                            print("Sorry I couldn't find an answer.")
                        print("Jarvis: ",end='')
                        print(response(query,sent_tokens)+'\n'+result[i])
                        
                    
                sent_tokens.remove(query)
                
    elif ('bye' in query.split(' ')):
        flag=False
        byee()

Jarvis: Hi! My name is Jarvis. I will answer all your queries. If you want to exit, type something like 'Bye' !
May I know your name?
Saloni
Jarvis: Hello  Saloni , How can I help you today?
Saloni: plural of emojis
Jarvis: He said the Associated Press began recommending emojis as the plural in 2014.
https://www.grammarly.com/blog/emojis-emoji/
Are you satisfied by my answer? Enter "yes" or "no".
no
Jarvis: Is the plural of 'emoji' 'emoji' or 'emojis'?
https://www.eveningexpress.co.uk/news/emoji-or-emojis-whats-the-correct-plural/
Are you satisfied by my answer? Enter "yes" or "no".
yes
Saloni: bye
Jarvis: Bye! take care..


### Okay, we need improvement here, but it works and finds information for us.

## But what more can we do?

The first and foremost thing for an A.I. assistant is to speak. To make our bot talk, we will code a speak() function. This function will take audio as an argument, and then, it will pronounce it.

In [19]:
!py -m pip install pyttsx3

You should consider upgrading via the 'C:\Users\Dr. Sunil Tripathi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


In [39]:
import pyttsx3
engine = pyttsx3.init('sapi5')
voice = engine.getProperty('voices') #getting details of the current voice
engine.setProperty('voice', voice[0].id)
def speak(audio):
    print(audio)
    engine.say(audio) 
    engine.runAndWait() #Without this command, speech will not be audible to us.

In [36]:
def introduce():
    #as we are taking a variable flag (true until user keeps asking questions)
    speak("Hi! My name is Jarvis. I will answer all your queries. If you want to exit, type something like 'Bye' !")
    return True
def byee():
    speak("Bye! take care..")
    return False

def qas():
    #Jarvis introduces itself
    flag=introduce()
    #site index
    speak('May I know your name?')
    #User introduces
    user_name=input()
    speak('Hello '+user_name+", How can I help you today?")

    while(flag==True):
        i=0
        #taking user question
        query = input(user_name+': ')
        query=query.lower()

        if('bye' not in query.split(' ')):
            if(query=='thanks' or query=='thank you' ):
                flag=False
                speak("You are welcome..")
            else:
                if(greeting(query)!=None):
                    speak(greeting(query))
                else:
                    result=webpageextraction(query)
                    raw=''
                    while((not raw.strip()) or (('access' in raw.lower()) and ('denied' in raw.lower()))):
                        raw=snippetextraction(result,i)
                        if raw=='error604':
                            break
                        i+=1
                    if raw=='error604':
                        speak("Sorry I couldn't find an answer.")
                        continue
                    if raw:
                        sent_tokens=tokenise(raw)
                    else:
                        speak("Sorry I couldn't find an answer.")
                    answer=response(query,sent_tokens)
                    if answer=='error502':
                        continue
                    speak(answer)
                    print('\n'+result[i])


                    #now checking satisfaction
                    satisfied=False
                    while(not satisfied):
                        speak('Are you satisfied by my answer? Enter "yes" or "no".')
                        if (input().lower()=='yes'):
                            break
                        else:
                            i+=1
                            raw=snippetextraction(result,i)
                            while((not raw.strip()) or (('access' in raw.lower()) and ('denied' in raw.lower()))):
                                raw=snippetextraction(result,i)
                                i+=1
                            if raw=='error604':
                                speak('Sorry, I am out of answers now.')
                                speak("Anything else you would like to ask?")
                                break
                            if raw:
                                sent_tokens=tokenise(raw)
                            else:
                                speak("Sorry I couldn't find an answer.")
                            speak(response(query,sent_tokens))
                            print('\n'+result[i])
                    speak("Is there anything else you would like to know?")


                    sent_tokens.remove(query)

        elif ('bye' in query.split(' ')):
            flag=False
            byee()

In [37]:
qas()

Hi! My name is Jarvis. I will answer all your queries. If you want to exit, type something like 'Bye' !
May I know your name?
Saloni
Hello Saloni, How can I help you today?
Saloni: Bye
Bye! take care..


In [14]:
qas()

Hi! My name is Karen. I will answer all your queries. If you want to exit, type something like 'Bye' !
May I know your name?
Saloni
Hello Saloni, How can I help you today?
Saloni: tell me something about farmer protest
I am sorry! I don't understand you
Saloni: farmer protests in India
Farm laws are death warrant for farmers: Arvind Kejriwal  Agitating farmers announce series of events between February 23 and 27 to intensify their protest against three Central agricultural laws  This (farmers' protest) is politically influenced.

https://en.wikipedia.org/wiki/2020%25E2%2580%25932021_Indian_farmers%2527_protest
Are you satisfied by my answer? Enter "yes" or "no".
no
Photo: Reuters/Danish Siddiqui New Delhi: Citing damning examples of Reagan era policies that have led to irreparable damage to the US’s farmers, 87 farmers’ unions in the country have extended solidarity to the ongoing protests by farmers in India.

https://thewire.in/agriculture/usa-farmers-unions-msp-protests-reagan
Are y

### Okay, so now it speaks, let us make it hear us.

In [20]:
!py -m pip install speechRecognition

You should consider upgrading via the 'C:\Users\Dr. Sunil Tripathi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


In [16]:
!py -m pip install PyAudio

You should consider upgrading via the 'C:\Users\Dr. Sunil Tripathi\AppData\Local\Programs\Python\Python38-32\python.exe -m pip install --upgrade pip' command.


In [13]:
import speech_recognition as sr
import pyaudio

In [27]:
def takeCommand():
    #It takes microphone input from the user and returns string output
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        print("Listening...")
        r.pause_threshold = 1
        audio = r.listen(source, timeout=5, phrase_time_limit=3)
        try:
            print("Recognizing...")    
            query = r.recognize_google(audio, language='en-in') #Using google for voice recognition.
            print(f"You said: {query}\n")  #User query will be printed.
        except Exception as e:
            # print(e)  use only if you want to print the error!
            print("Say that again please...")   #Say that again will be printed in case of improper voice 
            return "None" #None string will be returned
    return query

In [22]:
takeCommand()

Listening...
Recognizing...
You said: how are you today



'how are you today'

In [40]:
def qasWithVoiceInput():
    #Jarvis introduces itself
    flag=introduce()
    #site index
    speak('May I know your name?')
    #User introduces
    user_name=takeCommand()
    speak('Hello '+user_name+", How can I help you today?")

    while(flag==True):
        i=0
        #taking user question
        print(user_name+': ', end="")
        query = takeCommand()
        query=query.lower()

        if('bye' not in query.split(' ')):
            if(query=='thanks' or query=='thank you' ):
                flag=False
                speak("You are welcome..")
            else:
                if(greeting(query)!=None):
                    speak(greeting(query))
                else:
                    result=webpageextraction(query)
                    raw=''
                    while((not raw.strip()) or (('access' in raw.lower()) and ('denied' in raw.lower()))):
                        raw=snippetextraction(result,i)
                        if raw=='error604':
                            break
                        i+=1
                    if raw=='error604':
                        speak("Sorry I couldn't find an answer.")
                        continue
                    if raw:
                        sent_tokens=tokenise(raw)
                    else:
                        speak("Sorry I couldn't find an answer.")
                    answer=response(query,sent_tokens)
                    if answer=='error502':
                        continue
                    speak(answer)
                    print('\n'+result[i])


                    #now checking satisfaction
                    satisfied=False
                    while(not satisfied):
                        speak('Are you satisfied by my answer? Enter "yes" or "no".')
                        if (takeCommand().lower()=='yes'):
                            break
                        else:
                            i+=1
                            raw=snippetextraction(result,i)
                            while((not raw.strip()) or (('access' in raw.lower()) and ('denied' in raw.lower()))):
                                raw=snippetextraction(result,i)
                                i+=1
                            if raw=='error604':
                                speak('Sorry, I am out of answers now.')
                                speak("Anything else you would like to ask?")
                                break
                            if raw:
                                sent_tokens=tokenise(raw)
                            else:
                                speak("Sorry I couldn't find an answer.")
                            speak(response(query,sent_tokens))
                            print('\n'+result[i])
                    speak("Is there anything else you would like to know?")


                    sent_tokens.remove(query)

        elif ('bye' in query.split(' ')):
            flag=False
            byee()

In [ ]:
qasWithVoiceInput()

Hi! My name is Jarvis. I will answer all your queries. If you want to exit, type something like 'Bye' !
May I know your name?
Listening...
Recognizing...
You said: Saloni

Hello Saloni, How can I help you today?
Saloni: Listening...
Recognizing...
You said: tell me about rain

[25] Rain drops associated with melting hail tend to be larger than other rain drops.

https://simple.wikipedia.org/wiki/Rain
Are you satisfied by my answer? Enter "yes" or "no".
Listening...
Recognizing...
You said: no

I am sorry! I don't understand you
error502


In [ ]:
qasWithVoiceInput()

Hi! My name is Karen. I will answer all your queries. If you want to exit, type something like 'Bye' !
May I know your name?
Listening...
Recognizing...
You said: Saloni

Hello Saloni, How can I help you today?
Saloni: Listening...
Recognizing...
You said: tell me more about Triple IT Pune

Is there any placements at IIIT Pune?

https://collegedunia.com/university/57979-indian-institute-of-information-technology-iiit-pune/admission
Are you satisfied by my answer? Enter "yes" or "no".
Listening...
Recognizing...
You said: no

Read More IIIT Pune Placement Following are the facilities available in Indian Institute of Information Technology, Pune: Read More IIIT Pune Hostel Ques.

https://collegedunia.com/university/57979-indian-institute-of-information-technology-iiit-pune
Are you satisfied by my answer? Enter "yes" or "no".
Listening...
Recognizing...
You said: yes

Is there anything else you would like to know?
Saloni: Listening...
Recognizing...
You said: tell me something about farme

## Many many great features are coming soon...